https://avielrs.github.io/Intro-Market-Basket-Analysis/

https://www.kaggle.com/datasets/mkechinov/ecommerce-behavior-data-from-multi-category-store

https://github.com/nurlailiis/Association-Algorithm/blob/master/Dataset/control_data.csv

https://github.com/krishna-aditi/product-recommendation-based-on-market-basket-analysis-and-rfm-analysis

https://github.com/huytjuh/Recommender-System-Basket-Analysis

https://github.com/Sriram-infohub/Market-Basket-Analysis-and-Recommendation-System/blob/main/AdvML_4(Retail).ipynb

https://github.com/AjNavneet/Rule-Based-Recommendation-Market-Basket-Analysis-Retail

To extract data from greenbasket db

```sql
SELECT 
    oi.order_id as Transaction,
    MAX(CASE WHEN product_rank = 1 THEN p.name END) AS Product1,
    MAX(CASE WHEN product_rank = 2 THEN p.name END) AS Product2,
    MAX(CASE WHEN product_rank = 3 THEN p.name END) AS Product3,
    MAX(CASE WHEN product_rank = 4 THEN p.name END) AS Product4,
    MAX(CASE WHEN product_rank = 5 THEN p.name END) AS Product5,
    MAX(CASE WHEN product_rank = 6 THEN p.name END) AS Product6,
    MAX(CASE WHEN product_rank = 7 THEN p.name END) AS Product7,
    MAX(CASE WHEN product_rank = 8 THEN p.name END) AS Product8,
    MAX(CASE WHEN product_rank = 9 THEN p.name END) AS Product9,
    MAX(CASE WHEN product_rank = 10 THEN p.name END) AS Product10,
    MAX(CASE WHEN product_rank = 11 THEN p.name END) AS Product11,
    MAX(CASE WHEN product_rank = 12 THEN p.name END) AS Product12,
    MAX(CASE WHEN product_rank = 13 THEN p.name END) AS Product13,
    MAX(CASE WHEN product_rank = 14 THEN p.name END) AS Product14,
    MAX(CASE WHEN product_rank = 15 THEN p.name END) AS Product15,
    MAX(CASE WHEN product_rank = 16 THEN p.name END) AS Product16,
    MAX(CASE WHEN product_rank = 17 THEN p.name END) AS Product17,
    MAX(CASE WHEN product_rank = 18 THEN p.name END) AS Product18,
    MAX(CASE WHEN product_rank = 19 THEN p.name END) AS Product19,
    MAX(CASE WHEN product_rank = 20 THEN p.name END) AS Product20,
    MAX(CASE WHEN product_rank = 21 THEN p.name END) AS Product21,
    MAX(CASE WHEN product_rank = 22 THEN p.name END) AS Product22
FROM (
    SELECT 
        oi.order_id,
        oi.product_id,
        ROW_NUMBER() OVER (PARTITION BY oi.order_id ORDER BY oi.product_id) AS product_rank
    FROM order_items oi
) oi
JOIN products p ON p.id = oi.product_id
GROUP BY oi.order_id;
```

#### Imports Required Libraries

In [360]:
import pandas as pd
import numpy as np
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

#### Load dataset and print two rows

In [361]:
df = pd.read_csv("Market_Basket_Optimisation.csv")
df.head()

,item1,item2,item3,item4,item5,item6,item7
0,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams
1,burgers,meatballs,eggs,NaN,NaN,NaN,NaN
2,chutney,NaN,NaN,NaN,NaN,NaN,NaN
3,turkey,avocado,NaN,NaN,NaN,NaN,NaN
4,mineral water,milk,energy bar,whole wheat rice,green tea,NaN,NaN


In [362]:
df.shape

(7501, 7)

#### Print column names with total empty/NaN value counts

In [363]:
print("Column Name \t Total Null")
df.isnull().sum()

Column Name 	 Total Null


item1       0
item2    1754
item3    3112
item4    4156
item5    4972
item6    5637
item7    6132
dtype: int64

#### Replaces 'NaN' values with empty string

In [364]:
df.fillna("", inplace=True)
df.head()

,item1,item2,item3,item4,item5,item6,item7
0,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams
1,burgers,meatballs,eggs,,,,
2,chutney,,,,,,
3,turkey,avocado,,,,,
4,mineral water,milk,energy bar,whole wheat rice,green tea,,


In [365]:
for i in range(df.shape[1]):
    df[f"item{i + 1}"] = df[f"item{i + 1}"].apply(lambda word: word.strip())

#### Encode the dataset into true/false values using tansaction encoder

In [366]:
te = TransactionEncoder()
encoded_array  = te.fit_transform(df.values.tolist())
encoded_array

array([[False,  True, False, ...,  True, False, False],
       [ True, False, False, ..., False, False, False],
       [ True, False, False, ..., False, False, False],
       ...,
       [ True, False, False, ..., False, False, False],
       [ True, False, False, ..., False, False, False],
       [ True, False, False, ..., False,  True, False]])

#### print encoded array column names

In [367]:
len(te.columns_)

120


#### Convert the encoded array into pandas dataframe

In [368]:
transaction_df = pd.DataFrame(encoded_array, columns= te.columns_)
transaction_df.head()

,,almonds,antioxydant juice,asparagus,avocado,babies food,bacon,barbecue sauce,black tea,blueberries,...,turkey,vegetables mix,water spray,white wine,whole weat flour,whole wheat pasta,whole wheat rice,yams,yogurt cake,zucchini
0,False,True,False,False,True,False,False,False,False,False,...,False,True,False,False,True,False,False,True,False,False
1,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,True,False,False,False,True,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
4,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False


#### Extract frequent items by using apriori algorithm

In [369]:
frequent_items = apriori(transaction_df, min_support=0.01, use_colnames=True)
frequent_items

,support,itemsets
0,0.817491,()
1,0.018531,(almonds)
2,0.032529,(avocado)
3,0.013465,(black tea)
4,0.025597,(brownies)
...,...,...
259,0.012932,"(chocolate, mineral water, spaghetti)"
260,0.010399,"(eggs, mineral water, spaghetti)"
261,0.011332,"(frozen vegetables, mineral water, spaghetti)"
262,0.016398,"(mineral water, spaghetti, ground beef)"


#### Genreate association rules for frequent items

In [370]:
rules = association_rules(frequent_items, min_threshold=0.01)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(),(avocado),0.817491,0.032529,0.016798,0.020548,0.631681,1.0,-0.009794,0.987768,-0.761609,0.020160,-0.012384,0.268471
1,(avocado),(),0.032529,0.817491,0.016798,0.516393,0.631681,1.0,-0.009794,0.377391,-0.376046,0.020160,-1.649768,0.268471
2,(),(brownies),0.817491,0.025597,0.017198,0.021037,0.821874,1.0,-0.003727,0.995343,-0.542859,0.020823,-0.004679,0.346456
3,(brownies),(),0.025597,0.817491,0.017198,0.671875,0.821874,1.0,-0.003727,0.556218,-0.181953,0.020823,-0.797857,0.346456
4,(),(burgers),0.817491,0.087188,0.050260,0.061481,0.705149,1.0,-0.021016,0.972608,-0.696147,0.058824,-0.028163,0.318967
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
505,"(milk, spaghetti)",(mineral water),0.032396,0.237168,0.013332,0.411523,1.735150,1.0,0.005648,1.296280,0.437866,0.052029,0.228562,0.233867
506,"(mineral water, spaghetti)",(milk),0.058925,0.125717,0.013332,0.226244,1.799638,1.0,0.005924,1.129922,0.472155,0.077821,0.114983,0.166144
507,(milk),"(mineral water, spaghetti)",0.125717,0.058925,0.013332,0.106045,1.799638,1.0,0.005924,1.052709,0.508225,0.077821,0.050069,0.166144
508,(mineral water),"(milk, spaghetti)",0.237168,0.032396,0.013332,0.056211,1.735150,1.0,0.005648,1.025234,0.555405,0.052029,0.024613,0.233867


#### Print association rules with necessary columns

In [371]:
rules[["antecedents", "consequents", "support", "confidence"]]

,antecedents,consequents,support,confidence
0,(),(avocado),0.016798,0.020548
1,(avocado),(),0.016798,0.516393
2,(),(brownies),0.017198,0.021037
3,(brownies),(),0.017198,0.671875
4,(),(burgers),0.050260,0.061481
...,...,...,...,...
505,"(milk, spaghetti)",(mineral water),0.013332,0.411523
506,"(mineral water, spaghetti)",(milk),0.013332,0.226244
507,(milk),"(mineral water, spaghetti)",0.013332,0.106045
508,(mineral water),"(milk, spaghetti)",0.013332,0.056211
